In [1]:
import torch
from transformers import pipeline

# Download and Save the model

In [ ]:
# pipe = pipeline("text-classification", model="ProsusAI/finbert")
# pipe.save_pretrained("/data/finbert")

# Load Model

In [2]:
pipe = pipeline("text-classification", model="/data/finbert")

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


# Sample Predictions

In [3]:
sentences = "this stock is going to fall, I wont buy it"#, "this has some potential but I am still not confident"
pipe(sentences)

[{'label': 'negative', 'score': 0.8971117734909058}]

In [5]:
# We use a variant of ChatML to format each message
import time

t1 = time.time()

print("before model start")
# We use a special <|end|> token with ID 49155 to denote ends of a turn
outputs = pipe(sentences)

t2 = time.time()
t2-t1

before model start


0.040109872817993164

In [4]:
#!pip install transformers==4.34.1

In [6]:
from refractml import *
from refractml.constants import MLModelFlavours

# # new score functions
from mosaic_utils.ai.score.base import ScoreBase
from typing import Tuple, Union, List, Any
import numpy as np

/opt/conda/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [24]:
class ScoreTemplateExample(ScoreBase):
    """
    This Class Demonstrate How To Implements ScoreBase Interface Class And It Basic Usage.
    """    
    def __init__(self):
        super().__init__()

        self.model_loaded = None

        import torch
        from transformers import pipeline

        if self.model_loaded is None:
            print("LLM model loading from data section")
            self.model_loaded = pipeline("text-classification", model="/data/finbert")

    def request_processing_fn(self, request) :
        """
            Processes Request Object -> List[Input data, ..]. It could be:
                   A List Mapping of All Value Can Be one of : 
                       - List[ [Feature_Value1, Feature_Value2, ...], [...] ]
                       - List[numpy.array(), numpy.array(), ...]
                       - List[tf.Tensor, tf.Tensor, tf.Tensor, ...]
                       - List[ SingleSample, SingleSample]

            :return: (n_inputs, payload's)

            Warnings:
            1. Do not reshape your final output for single sample here, do it in prediction.
               Else payloads will be invalidated for extraction at raw and extraction level.
            """
        final_payload = []
        raw_payload = request.json["payload"]
        return (1, raw_payload) 

    def pre_processing_fn(self,payload):
        
        #         # All preprocessing step must occur in this section
        #         # Takes Single Sample -> Returns Single Sample

        #         # Not Doing Any Preprocessing Hence Returned payload
        #         print("payload is ", payload)
        return payload

    def prediction_fn(self,model: Any,input_query):
        """
                Does the main prediction on pre_processed_input(Single Sample) using supplied model .

                :param model: Supported Model
                :param pre_processed_input: Single Preprocessed Payload
                :return: Prediction Value From the model

                Important Notes:
                - Reshape your data array.reshape(1, -1) before predictions as it contains a single sample.

        """
        model_loaded = self.model_loaded
        mod = model_loaded
        text = pre_processed_input #this is tuple we can iterate if there is number of input


        preds = pipe(input_query)
        print("prediction is \n: ",preds)

        return preds

    #     class Meta:    
         # List of Callables() can be attached For Calling After AnSd Before Scoring
    #         def __init__(self):
    #             self.name = "Pre Hooked Me !"
    #             self.pre_call_hooks.append(self.print_)
    #         def print_(self):
    #             print(self.name)
    #             pre_call_hooks = []
    #             post_call_hooks = []

In [27]:
score_ = ScoreTemplateExample()
import requests
req = requests.Request()

req.json = {"payload":"['this stock is going to fall']"}

LLM model loading from data section


In [28]:
t3 = time.time()
model_predictions = score_.score(None, req, dry_run=True)
t4 = time.time()
t4-t3

,VALIDATION,COMPONENT,PASSED,SKIPPED
0,"Return Type Must Be Tuple (n_input, payloads)",request_processing_fn,True,False
1,"Tuple Must Be of length Two (n_input, payloads)",request_processing_fn,True,False
2,"* if n_input > 1 payload type must be List (n_input, [np.ndarray, tf.Tensor, etc])",request_processing_fn,True,False


Fields Marked Asterisk (*) Can Be Validated On Proper Input 



NameError: name 'pre_processed_input' is not defined

In [9]:
print(model_predictions)

[<<ScoreResponse>>]


In [10]:
model="/data/huggingface/cache/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f"

In [ ]:
register_model(model,
               ScoreTemplateExample,
               "Starchat_Beta",
               "Starchat_Beta_for_code_Generation",
               MLModelFlavours.pytorch,
               init_script="pip install SentencePiece \\n pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu"
              )

INFO:root:PASS:: Mandatory Validation :: dict_keys(['name', 'description', 'flavour', 'scoring_func'])
INFO:root:PASS:: AlphaNumeric Validation :: dict_keys(['name'])
INFO:root:PASS:: Validation/IfPresentTypeCheck :: dict_keys(['schema', 'metadata_info', 'tags'])
INFO:root:PASS:: Validation/IfPresentSubfieldMustExist :: dict_keys(['kyd'])
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): mosaic-ai-backend:5000
